# FMCA interface

### first import modules

In [1]:
# import seems necessary to not crash matplotlib
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import linalg as la
import time
import FMCA

In [2]:
dim = 3
N = 1000000
ridge = 1e-3
rhs = np.ones((N,1), order='F')
cov = FMCA.CovarianceKernel("GAUSSIAN", 1.)
pts = np.array(np.random.rand(dim, N), order='F')

In [3]:
chol = FMCA.PivotedCholesky();
falkon = FMCA.FALKON();

Based on the Woodbury matrix identity, we have
$$
B = A+UV\Rightarrow B^{-1}=A^{-1}-A^{-1}U(I+VA^{-1}U)^{-1}VA^{-1}.
$$
Setting $A = n\lambda I$, $U=L$ and $V=L^T$, this yields
$$
(n\lambda I + LL^T)^{-1}=\frac{1}{n\lambda}I-\frac{1}{n\lambda}IL\bigg(I+L^T\frac{1}{n\lambda}IL\bigg)^{-1}L^T\frac{1}{n\lambda}I.
$$
This simplifies to
$$
(n\lambda I + LL^T)^{-1}=\frac{1}{n\lambda}(I-L(n\lambda I+L^TL)^{-1}L^T).
$$
We get to a FALKON like low-rank representation of the coefficient $\alpha$ by setting $\beta=L^T\alpha$. We arrive at
$$
(n\lambda I + LL^T)\alpha=f\Rightarrow \alpha = \frac{1}{n\lambda}(I-L(n\lambda I+L^TL)^{-1}L^T)f
\Rightarrow\beta=\frac{1}{n\lambda}(L^T-L^TL(n\lambda I+L^TL)^{-1}L^T)f
$$
and
$$
f\approx L\beta
$$

In [ ]:
tchol = np.zeros((20,1))
errchol = np.zeros((20,1))
tfalkon = np.zeros((20,1))
errfalkon = np.zeros((20,1))
for i in range(20):
    prec = 2**(-1-i)
    print('prec: ', prec)
    # pivoted Cholesky
    start = time.time()
    chol.compute(cov, pts, prec);
    LTL = chol.matrixL().transpose() @ chol.matrixL()
    M = (LTL.shape)[1]
    LTrhs = chol.matrixL().transpose() @ rhs
    beta = (LTrhs - LTL @ np.linalg.solve(N * ridge * np.identity(M) + LTL, LTrhs)) / (N * ridge);
    stop = time.time()
    tchol[i] = stop - start
    errchol[i] = np.linalg.norm((chol.matrixL() @ beta).flatten() - rhs.flatten()) / np.linalg.norm(rhs.flatten())
    print('elapsed time Cholesky computation: ', tchol[i], 'sec.')
    print('error: ', errchol[i])
    # FALKON (using pivoted Cholesky rank)
    start = time.time()
    falkon.init(cov, pts, M, ridge)
    alpha = falkon.computeAlpha(rhs, 10)
    stop = time.time()
    tfalkon[i] = stop - start
    errfalkon[i] = np.linalg.norm((falkon.matrixKPC() @ alpha).flatten() - rhs.flatten()) / np.linalg.norm(rhs.flatten())
    print('elapsed time FALKON init/solution: ', tfalkon[i], 'sec.')
    print('error: ', errfalkon[i])

prec:  0.5
N: 1000000 max number of cols: 1000
rel tol: 500000 initial trace: 1e+06
steps: 1 trace error: 331341
elapsed time Cholesky computation:  [1.34004498] sec.
error:  [0.13442859]
CG iterations: 1 relative residual: 0
elapsed time FALKON init/solution:  [0.47449303] sec.
error:  [0.16775615]
prec:  0.25
N: 1000000 max number of cols: 1000
rel tol: 250000 initial trace: 1e+06
steps: 2 trace error: 202986
elapsed time Cholesky computation:  [1.75306916] sec.
error:  [0.07649405]
CG iterations: 3 relative residual: 1.95316e-17
elapsed time FALKON init/solution:  [0.92744708] sec.
error:  [0.06282029]
prec:  0.125
N: 1000000 max number of cols: 1000
rel tol: 125000 initial trace: 1e+06
steps: 4 trace error: 85833.3
elapsed time Cholesky computation:  [2.63790298] sec.
error:  [0.05540678]
CG iterations: 8 relative residual: 3.18527e-26
elapsed time FALKON init/solution:  [1.98138881] sec.
error:  [0.06195644]
prec:  0.0625
N: 1000000 max number of cols: 1000
rel tol: 62500 initial 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(20), tchol, 'bo-', label='pivChol')
plt.plot(range(20), tfalkon, 'ro-', label='FALKON')
plt.xlabel("step")
plt.ylabel("time (s)")
plt.legend()
plt.show()
plt.figure(figsize=(15,5))
plt.semilogy(tchol,errchol, 'bo-', label='pivChol')
plt.semilogy(tfalkon, errfalkon, 'ro-', label='FALKON')
plt.xlabel("time (s)")
plt.ylabel("error")
plt.legend()
plt.show()